In [ ]:
import logging
import sys
from dotenv import load_dotenv

load_dotenv(".env")
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [ ]:
from llama_index import SimpleDirectoryReader

# ドキュメントの読み込み
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
from llama_index import GPTVectorStoreIndex, ServiceContext, PromptHelper, LLMPredictor, LangchainEmbedding
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

# インデックスの作成
# index = GPTVectorStoreIndex.from_documents(documents)

# 使用するLLMをgpt-3.5-turboに変更（デフォルトはtext-davinci-003）
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

# 埋め込みモデルの準備（モデルはhugging faceなどからダウンロードする）
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"))

# チャンク分割ルールのカスタマイズ
prompt_helper = PromptHelper(max_input_size=4096, num_output=256, max_chunk_overlap=20)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    prompt_helper=prompt_helper,
    embed_model=embed_model,
)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
# クエリエンジンの作成
query_engine = index.as_query_engine()

# print(query_engine.query("ミコの幼馴染の名前は？"))
# print(query_engine.query("ウルフコーポレーションのCEOの名前は？"))
print(query_engine.query("ミコの性格は？"))

In [ ]:
# インデックスの保存
index.storage_context.persist()

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# インデックスの読み込み
storage_context = StorageContext.from_defaults(persist_dir="./storage")
idnex = load_index_from_storage(storage_context)